# Download audio file


In [181]:
import youtube_dl
#from youtube_dl import YoutubeDL
import re
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import pytube
from moviepy.editor import *
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import ffmpeg
import subprocess
from python_speech_features import mfcc
from python_speech_features import fbank
from python_speech_features import logfbank
import scipy.io.wavfile as wav

In [179]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5887 sha256=847d304252142df419ae14db792c799c3a1f6f9832b70b99cde06eafd46f7cf9
  Stored in directory: /Users/andrea/Library/Caches/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python-speech-features


## Create a pandas dataset

In [3]:
df=pd.read_csv(os.path.abspath(os.path.join(os.getcwd(),os.pardir,os.pardir))+'/data/data_split.csv',index_col=0)
df.head() 

,url,start_time,end_time,class_label
0,SpNYhh5nwBk,50.0,60.0,Animal
1,Y-5QMgGV1H0,0.0,10.0,Animal
2,2yRlkPqa6rQ,20.0,30.0,Animal
3,NYiIU62GHV4,30.0,40.0,Animal
4,x3c0szZ7VYI,60.0,70.0,Animal


In [36]:
df['class_label'].value_counts()

Natural    5000
Humans     5000
Animal     5000
Name: class_label, dtype: int64

In [75]:
for i, row in tqdm(df.iterrows()):
    print(row)
    break

0it [00:00, ?it/s]

url            SpNYhh5nwBk
start_time              50
end_time                60
class_label         Animal
Name: 0, dtype: object


## Download Audio \& extract features


In [245]:

def get_audio_features(filename, select='mfcc'):
    nfft = 513
    cases = {'mfcc':mfcc, 'fbank':fbank}
    
    #'logfbank':logfbank, delta, etc
    if select not in cases.keys():
        return 0
    
    (rate,sig) = wav.read(filename+".wav")
    
    return cases[select](sig,rate,nfft=nfft,winfunc=np.hamming)

In [239]:
{'mfcc':mfcc, 'fbank':fbank}.values()

dict_values([<function mfcc at 0x7fbe594784d0>, <function fbank at 0x7fbe5e63edd0>])

In [231]:
import warnings; warnings.simplefilter('ignore')

In [ ]:

for i, row in tqdm(df.iterrows()):
    if i==5:#download first 10 .wav files
        break
    url = 'https://www.youtube.com/watch?v=' + row['url']
    file_name = str(i)+"_"+row['class_label']
    
    
    try:
        
        #download youtube video & create a clipped .wav file
        subprocess.Popen("ffmpeg -ss " + str(row['start_time']) + " -i $(youtube-dl -f 140 --get-url " + url + ") -t 10 -c:v copy -c:a copy " + file_name + ".mp4", shell=True).wait()
        subprocess.Popen("ffmpeg -i "+file_name+".mp4 -ab 160k -ac 2 -ar 44100 -vn "+file_name+'.wav',shell=True).wait()
        
        #extract and save mfcc features to .npy
        features = get_audio_features(file_name)
        #add other features ?
        np.save(file_name,features)
        
        #remove .mp4 and .wav files
        subprocess.Popen('rm '+file_name+'.mp4',shell=True).wait()
        subprocess.Popen('rm '+file_name+'.wav',shell=True).wait()


    except:
        print('Aladin', url)
        pass
    

0it [00:00, ?it/s]WARNING:root:frame length (1103) is greater than FFT size (513), frame will be truncated. Increase NFFT to avoid.
1it [00:06,  6.22s/it]WARNING:root:frame length (1103) is greater than FFT size (513), frame will be truncated. Increase NFFT to avoid.
3it [00:15,  5.22s/it]

Aladin https://www.youtube.com/watch?v=2yRlkPqa6rQ


4it [00:21,  5.65s/it]

citare: https://github.com/HareeshBahuleyan/music-genre-classification/blob/master/1_audio_retrieval.ipynb